In [ ]:
# @title Setup (Deps)
try:
  from google.colab import drive, userdata
  if not os.path.exists("owl2vec_star"):
    !rm -rf * .config
    !git clone https://github.com/mmtftr/owl2vec_star.git .

  import os

  wandb_api_key = userdata.get('WANDB_API_KEY')
  drive.mount('/content/drive')
  os.environ['WANDB_API_KEY'] = wandb_api_key
  if not os.path.exists("/content/drive/MyDrive/research/GO_emb"):
    raise Exception("GO_emb directory not found in drive")

  !cp -r /content/drive/MyDrive/research/GO_emb/* .
  !unzip go-basic.zip -d go-basic
  !unzip go-full.zip -d go-full
  %pip install -r requirements_owl2vec.txt
  %cd mmtf-eval
except:
  device = "mps" if torch.mps.is_available() else "cpu"

In [ ]:

# @title Configuration
# @arg base_ontology: "go-basic" or "go-full"
# @arg embedding_types: list of embedding types to evaluate
# @arg models: list of models to evaluate
# @arg input_type: "concatenate" or "minus"
# @arg random_seed: random seed
base_ontology = "go-basic"
embedding_types = ["owl2vec"]
models = ["mlp"]
input_type = "concatenate"
random_seed = 42

emb_types_str = ", ".join([f"{emb_type}" for emb_type in embedding_types])
models_str = ", ".join([f"{model}" for model in models])

config = f"""
# Project settings
project_name: "go-embedding-evaluation"
base_ontology: {base_ontology}
embedding_types: [{emb_types_str}]
models: [{models_str}]
input_type: {input_type}
random_seed: {random_seed}


# Training parameters
batch_size: 128
learning_rate: 0.001
num_epochs: 100
"""

with open("config.yaml", "w") as f:
  f.write(config)


In [ ]:
!python embedding_evaluation.py